In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import xarray as xr
import xskillscore as xs
import skimage
import pandas as pd
import seaborn as sns
import math
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils"
os.chdir(path)
import metrics_gev
import dataset_copy
import rescaling
import result_gev

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from scipy.interpolate import interpolate
import xarray as xr
import xesmf as xe

In [4]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data"
os.chdir(path)

In [5]:
aout=xr.open_dataset("aout_10_premier_jour.nc")
lat_bnd = [-0.6, 0.58]
lon_bnd = [-2.6,-1.42]
aout.sel(rlon=slice(*lon_bnd), rlat=slice(*lat_bnd))

<xarray.Dataset>
Dimensions:     (rlat: 60, rlon: 60, time: 240)
Coordinates:
    lon         (rlat, rlon) float32 ...
    lat         (rlat, rlon) float32 ...
  * rlon        (rlon) float32 -2.6 -2.58 -2.56 -2.54 ... -1.46 -1.44 -1.42
  * rlat        (rlat) float32 -0.6 -0.58 -0.56 -0.54 ... 0.52 0.54 0.56 0.58
  * time        (time) datetime64[ns] 2003-08-02 ... 2003-08-11T23:00:00
    height_2m   float32 ...
    height_10m  float32 ...
Data variables:
    T_2M        (time, rlat, rlon) float32 ...
    RELHUM_2M   (time, rlat, rlon) float32 ...
    TOT_PR      (time, rlat, rlon) float32 ...
Attributes:
    standard_name:  air_temperature
    long_name:      2m temperature
    units:          K
    grid_mapping:   rotated_pole

In [6]:
df = pd.read_csv('gev_data_juillet_aout.csv')
df

,rlon,rlat,shape,scale,loc
0,-2.60,-0.60,0.516177,0.002319,0.005034
1,-2.60,-0.58,0.214582,0.003072,0.004107
2,-2.60,-0.56,0.431718,0.003952,0.004903
3,-2.60,-0.54,0.016592,0.002638,0.003848
4,-2.60,-0.52,0.395072,0.002790,0.004489
...,...,...,...,...,...
3595,-1.42,0.50,0.224239,0.001944,0.004947
3596,-1.42,0.52,0.395784,0.003286,0.004591
3597,-1.42,0.54,0.183072,0.004131,0.005818
3598,-1.42,0.56,0.075979,0.003763,0.005709


In [7]:
time_value = ["08-01"]
gev_param = xr.DataArray(df['loc'].values.reshape((len(df['rlat'].unique()), len(df['rlon'].unique()))),
                  coords={'rlat': df['rlat'].unique(), 'rlon': df['rlon'].unique()},
                  dims=['rlat', 'rlon'])
gev_param = gev_param.expand_dims({'time': [time_value]})
# Print the resulting DataArray
print(gev_param)
gev_param = gev_param.to_dataset(name='location')
gev_param

<xarray.DataArray (time: 1, rlat: 60, rlon: 60)>
array([[[0.00503419, 0.00410715, 0.00490255, ..., 0.00501639,
         0.00551327, 0.00455693],
        [0.00613498, 0.00445625, 0.00373258, ..., 0.00407124,
         0.00426827, 0.00489673],
        [0.00565273, 0.00373708, 0.00584568, ..., 0.00558508,
         0.00465733, 0.00415159],
        ...,
        [0.00374745, 0.00672859, 0.00531398, ..., 0.00542053,
         0.0062348 , 0.00635421],
        [0.00577003, 0.00431323, 0.00630985, ..., 0.00537127,
         0.0054779 , 0.00564207],
        [0.00472048, 0.00573401, 0.00553599, ..., 0.00581821,
         0.00570942, 0.00357379]]])
Coordinates:
  * time     (time) object ('08-01',)
  * rlat     (rlat) float64 -0.6 -0.58 -0.56 -0.54 -0.52 ... 0.52 0.54 0.56 0.58
  * rlon     (rlon) float64 -2.6 -2.58 -2.56 -2.54 ... -1.48 -1.46 -1.44 -1.42


<xarray.Dataset>
Dimensions:   (time: 1, rlat: 60, rlon: 60)
Coordinates:
  * time      (time) object ('08-01',)
  * rlat      (rlat) float64 -0.6 -0.58 -0.56 -0.54 ... 0.52 0.54 0.56 0.58
  * rlon      (rlon) float64 -2.6 -2.58 -2.56 -2.54 ... -1.48 -1.46 -1.44 -1.42
Data variables:
    location  (time, rlat, rlon) float64 0.005034 0.004107 ... 0.005709 0.003574

In [8]:
#gev_param.plot()

In [8]:
ingrid = rescaling.create_grid(-2.6, -1.42, 0.02, -0.6, 0.6, 0.02)
outgrid = rescaling.create_grid(-2.6, -1.42, 0.12, -0.6, 0.6, 0.12)

In [10]:
##Besoin de faire un carré :

In [9]:
ups_ds = rescaling.upscale(gev_param, ingrid, outgrid)
ups_ds

<xarray.Dataset>
Dimensions:   (time: 1, rlat: 10, rlon: 10)
Coordinates:
  * time      (time) object ('08-01',)
  * rlon      (rlon) float64 -2.6 -2.48 -2.36 -2.24 ... -1.88 -1.76 -1.64 -1.52
  * rlat      (rlat) float64 -0.6 -0.48 -0.36 -0.24 ... 0.12 0.24 0.36 0.48
Data variables:
    location  (time, rlat, rlon) float64 0.005011 0.005118 ... 0.004892 0.00508
Attributes:
    regrid_method:  conservative

In [12]:
##On est obligés d'avoir un carré

In [10]:
def downscale_without_time(in_ds, target_ds, lon_min, lon_max, lat_min, lat_max, new_lon_max, new_lat_max, ups_dim, new_dim, interp_type): 
    #target_ds must be a dataset with the dimensions we want, and the same variables. It will be overwritten 
    # interp_type can be {'linear', 'cubic', 'quintic'}
    lon = np.arange(lon_min, lon_max, ups_dim)
    print(lon.shape)
    lat = np.arange(lat_min, lat_max, ups_dim)
    print(lat.shape)
    
    """   
    # Scale interpolation
    fun = interpolate.interp2d(lon, lat, in_ds.scale, kind=interp_type) 
    newlon = np.arange(lon_min, new_lon_max, new_dim)
    newlat = np.arange(lat_min, new_lat_max, new_dim)
    newvar = fun(newlon, newlat)
    target_ds['scale'] = (("lat", "lon"), newvar)
    
    # Shape interpolation
    fun = interpolate.interp2d(lon, lat, in_ds.shape, kind=interp_type)

    newlon = np.arange(lon_min, new_lon_max, new_dim)      
    newlat = np.arange(lat_min, new_lat_max, new_dim)
    newvar = fun(newlon, newlat)
    target_ds['shape'] = (("lat", "lon"), newvar)
    """       
    # Location interpolation
    fun = interpolate.interp2d(lon, lat, in_ds.location, kind=interp_type)
    newlon = np.arange(lon_min, new_lon_max, new_dim)
    newlat = np.arange(lat_min, new_lat_max, new_dim)
    newvar = fun(newlon, newlat)
    target_ds['location'] = (("rlat", "rlon"), newvar)        
    return target_ds


In [11]:
bilin_2km = xr.full_like(gev_param, 0)

downscale_without_time(ups_ds, bilin_2km, -2.6, -1.42, -0.6, 0.6, -1.42, 0.6, 0.12, 0.02, 'linear') #downscale(in_ds, target_ds, lon_min, lon_max, lat_min, lat_max, new_lon_max, new_lat_max, ups_dim, new_dim, interp_type): 

(10,)
(10,)


<xarray.Dataset>
Dimensions:   (rlat: 60, rlon: 60, time: 1)
Coordinates:
  * time      (time) object ('08-01',)
  * rlat      (rlat) float64 -0.6 -0.58 -0.56 -0.54 ... 0.52 0.54 0.56 0.58
  * rlon      (rlon) float64 -2.6 -2.58 -2.56 -2.54 ... -1.48 -1.46 -1.44 -1.42
Data variables:
    location  (rlat, rlon) float64 0.005011 0.005029 ... 0.00508 0.00508

In [12]:
bicubic_2km = xr.full_like(gev_param, 0)
downscale_without_time(ups_ds, bicubic_2km, -2.6, -1.42, -0.6, 0.6, -1.42, 0.6, 0.12, 0.02, 'cubic')

(10,)
(10,)


<xarray.Dataset>
Dimensions:   (rlat: 60, rlon: 60, time: 1)
Coordinates:
  * time      (time) object ('08-01',)
  * rlat      (rlat) float64 -0.6 -0.58 -0.56 -0.54 ... 0.52 0.54 0.56 0.58
  * rlon      (rlon) float64 -2.6 -2.58 -2.56 -2.54 ... -1.48 -1.46 -1.44 -1.42
Data variables:
    location  (rlat, rlon) float64 0.005011 0.005069 ... 0.00508 0.00508

In [13]:
quintic_2km = xr.full_like(gev_param, 0)
downscale_without_time(ups_ds, quintic_2km, -2.6, -1.42, -0.6, 0.6, -1.42, 0.6, 0.12, 0.02, 'quintic')

(10,)
(10,)


<xarray.Dataset>
Dimensions:   (rlat: 60, rlon: 60, time: 1)
Coordinates:
  * time      (time) object ('08-01',)
  * rlat      (rlat) float64 -0.6 -0.58 -0.56 -0.54 ... 0.52 0.54 0.56 0.58
  * rlon      (rlon) float64 -2.6 -2.58 -2.56 -2.54 ... -1.48 -1.46 -1.44 -1.42
Data variables:
    location  (rlat, rlon) float64 0.005011 0.005097 ... 0.00508 0.00508

In [14]:
methods = [0, 0, 0]
methods[0] = bilin_2km
methods[1] = bicubic_2km
methods[2] = quintic_2km

ds_ = result_gev.get_results(gev_param, ups_ds, methods, ["bilinear", "bicubic", "quintic"], 'location', [2, 2, 2, 12])

Frozen({'time': 1, 'rlat': 60, 'rlon': 60})
Frozen({'rlat': 60, 'rlon': 60, 'time': 1})
err Hellinger <xarray.Dataset>
Dimensions:   (time: 1)
Coordinates:
  * time      (time) object ('08-01',)
Data variables:
    location  (time) float64 nan
Frozen({'time': 1, 'rlat': 60, 'rlon': 60})
Frozen({'rlat': 60, 'rlon': 60, 'time': 1})
err Hellinger <xarray.Dataset>
Dimensions:   (time: 1)
Coordinates:
  * time      (time) object ('08-01',)
Data variables:
    location  (time) float64 nan
Frozen({'time': 1, 'rlat': 60, 'rlon': 60})
Frozen({'rlat': 60, 'rlon': 60, 'time': 1})
err Hellinger <xarray.Dataset>
Dimensions:   (time: 1)
Coordinates:
  * time      (time) object ('08-01',)
Data variables:
    location  (time) float64 nan


In [15]:
ds_

,method,RMSE,MAE,Hellinger,Perkins,mean
0,bilinear,0.999147,0.999324,NaN,NaN,0.999236
1,bicubic,0.999145,0.999323,NaN,NaN,0.999234
2,quintic,0.999138,0.999318,NaN,NaN,0.999228


In [ ]:
##Calculer le CRPS avec les nouveaux résultats en prenant shape et scale dans le cas stationnaire : 

In [16]:
def CRPS_calcul_interpolation(location,gev_stat,y_true):
    
        location = location.unsqueeze(-1).repeat(1, 1,22)        

        ##Tout de taille [60,60,22]
    
        c = torch.tensor(gev_stat['shape'])
        c = c.reshape(60, 60)
        c = torch.transpose(c, 0,1) ##oujours 60,60
        c = c.repeat(22,1,1)
        c = torch.permute(c,(1,2,0))
        scale = torch.tensor(gev_stat['scale'])
        scale = scale.reshape(60, 60)
        scale = torch.transpose(scale, 0, 1)  # Toujours 60, 60
        scale = scale.repeat(22, 1, 1)
        scale = torch.permute(scale, (1, 2, 0))

        size=(60,60,22)
        u = torch.rand(size)
            
        
        y_pred = location + (torch.pow(-torch.log(u), -c) - 1) * scale / c
    
        term_one = torch.mean(torch.abs(y_pred - y_true), dim=-1)
        term_two = torch.mean(torch.abs(
        torch.unsqueeze(y_pred, -1) - torch.unsqueeze(y_pred, -2)), dim=(-2, -1))
        half = torch.tensor(-0.5, dtype=term_two.dtype)
        loss = term_one + half * term_two
        print(loss)
        loss = torch.mean(loss)
        return loss

In [17]:
##On s'intéresse à juillet août : 
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils"
os.chdir(path)
import dataset_for_R 
import torch

out_ja = dataset_for_R.get_precipitation_xarray('juillet','aout',lon_bnd=(-2.6, -1.42),lat_bnd=(-0.6, 0.58))
out_ja = torch.tensor(out_ja.values)
out_ja = torch.flatten(out_ja,end_dim=1)
out_ja = torch.permute(out_ja,(1,2,0))

CRPS=[]

In [18]:
q = torch.tensor(quintic_2km.location.values)
bic = torch.tensor(bicubic_2km.location.values)
bil = torch.tensor(bilin_2km.location.values)
q.size()

torch.Size([60, 60])

In [19]:
gev_stat = pd.read_csv('gev_data_stationnary.csv')

In [20]:
CRPS_calcul_interpolation(bic,gev_stat,out_ja)
CRPS_calcul_interpolation(bil,gev_stat,out_ja)
CRPS_calcul_interpolation(q,gev_stat,out_ja)

tensor([[0.0023, 0.0025, 0.0028,  ..., 0.0019, 0.0009, 0.0017],
        [0.0022, 0.0027, 0.0020,  ..., 0.0024, 0.0032, 0.0017],
        [0.0015, 0.0026, 0.0031,  ..., 0.0016, 0.0017, 0.0026],
        ...,
        [0.0029, 0.0040, 0.0062,  ..., 0.0029, 0.0036, 0.0027],
        [0.0032, 0.0026, 0.0032,  ..., 0.0029, 0.0026, 0.0026],
        [0.0022, 0.0027, 0.0025,  ..., 0.0022, 0.0036, 0.0033]],
       dtype=torch.float64)
tensor([[0.0028, 0.0031, 0.0021,  ..., 0.0023, 0.0014, 0.0019],
        [0.0020, 0.0031, 0.0033,  ..., 0.0035, 0.0031, 0.0013],
        [0.0022, 0.0025, 0.0029,  ..., 0.0017, 0.0017, 0.0013],
        ...,
        [0.0020, 0.0043, 0.0048,  ..., 0.0027, 0.0031, 0.0035],
        [0.0032, 0.0030, 0.0032,  ..., 0.0021, 0.0025, 0.0026],
        [0.0020, 0.0032, 0.0007,  ..., 0.0030, 0.0033, 0.0019]],
       dtype=torch.float64)
tensor([[0.0024, 0.0025, 0.0027,  ..., 0.0026, 0.0011, 0.0023],
        [0.0024, 0.0017, 0.0017,  ..., 0.0021, 0.0037, 0.0017],
        [0.0016, 0.0

tensor(0.0031, dtype=torch.float64)

In [ ]:
CRPS_calcul_interpolation(bic,gev_stat,out_ja)

In [113]:
#Maintenant on récupère les données du modèle stat : 
loc = pd.read_csv('location_juillet_aout.csv')
loc = loc['x']
loc = torch.tensor(loc)
loc = loc.reshape(60, 60)
CRPS_calcul_interpolation(loc,gev_stat,out_ja)

torch.Size([60, 60, 22])
tensor([[0.0020, 0.0024, 0.0024,  ..., 0.0022, 0.0016, 0.0021],
        [0.0025, 0.0026, 0.0027,  ..., 0.0016, 0.0033, 0.0017],
        [0.0013, 0.0026, 0.0026,  ..., 0.0020, 0.0022, 0.0021],
        ...,
        [0.0027, 0.0046, 0.0056,  ..., 0.0026, 0.0034, 0.0027],
        [0.0012, 0.0032, 0.0037,  ..., 0.0018, 0.0024, 0.0028],
        [0.0028, 0.0032, 0.0027,  ..., 0.0029, 0.0022, 0.0029]],
       dtype=torch.float64)


tensor(0.0031, dtype=torch.float64)

In [115]:
def CRPS_calcul_interpolation_constants(location,c,scale,y_true):
    
        location = location.unsqueeze(-1).repeat(1, 1,22)        

        size=(60,60,22)
        u = torch.rand(size)
            
        
        y_pred = location + (torch.pow(-torch.log(u), -c) - 1) * scale / c
    
        term_one = torch.mean(torch.abs(y_pred - y_true), dim=-1)
        term_two = torch.mean(torch.abs(
        torch.unsqueeze(y_pred, -1) - torch.unsqueeze(y_pred, -2)), dim=(-2, -1))
        half = torch.tensor(-0.5, dtype=term_two.dtype)
        loss = term_one + half * term_two
        print(loss)
        loss = torch.mean(loss)
        return loss

In [123]:
scale = 0.003193901
c = 0.2120344
print('bilinear',CRPS_calcul_interpolation_constants(bil,c,scale,out_ja))
print('bicubic',CRPS_calcul_interpolation_constants(bic,c,scale,out_ja))
print('quintic',CRPS_calcul_interpolation_constants(q,c,scale,out_ja))
CRPS_calcul_interpolation_constants(loc,c,scale,out_ja)

tensor([[0.0026, 0.0031, 0.0023,  ..., 0.0024, 0.0016, 0.0025],
        [0.0023, 0.0031, 0.0024,  ..., 0.0033, 0.0035, 0.0016],
        [0.0022, 0.0028, 0.0013,  ..., 0.0007, 0.0033, 0.0030],
        ...,
        [0.0035, 0.0038, 0.0058,  ..., 0.0022, 0.0028, 0.0032],
        [0.0032, 0.0031, 0.0023,  ..., 0.0023, 0.0021, 0.0017],
        [0.0012, 0.0028, 0.0030,  ..., 0.0029, 0.0021, 0.0030]],
       dtype=torch.float64)
bilinear tensor(0.0029, dtype=torch.float64)
tensor([[0.0021, 0.0034, 0.0020,  ..., 0.0024, 0.0017, 0.0020],
        [0.0024, 0.0026, 0.0020,  ..., 0.0022, 0.0023, 0.0013],
        [0.0021, 0.0026, 0.0023,  ..., 0.0016, 0.0018, 0.0010],
        ...,
        [0.0022, 0.0051, 0.0057,  ..., 0.0032, 0.0028, 0.0024],
        [0.0037, 0.0028, 0.0032,  ..., 0.0021, 0.0030, 0.0023],
        [0.0022, 0.0026, 0.0030,  ..., 0.0023, 0.0032, 0.0023]],
       dtype=torch.float64)
bicubic tensor(0.0029, dtype=torch.float64)
tensor([[0.0026, 0.0036, 0.0024,  ..., 0.0020, 0.0019, 0.00

tensor(0.0029, dtype=torch.float64)

In [122]:
print(torch.mean(q))
torch.mean(loc)

tensor(0.0050, dtype=torch.float64)


tensor(0.0049, dtype=torch.float64)

In [125]:
loc = pd.read_csv('location_juillet_aout_splines.csv')
loc = loc['x']
loc = torch.tensor(loc)
loc = loc.reshape(60, 60)
CRPS_calcul_interpolation_constants(loc,c,scale,out_ja)

tensor([[0.0007, 0.0028, 0.0020,  ..., 0.0023, 0.0014, 0.0022],
        [0.0024, 0.0037, 0.0021,  ..., 0.0036, 0.0026, 0.0014],
        [0.0022, 0.0019, 0.0032,  ..., 0.0011, 0.0027, 0.0015],
        ...,
        [0.0026, 0.0037, 0.0054,  ..., 0.0019, 0.0023, 0.0029],
        [0.0031, 0.0030, 0.0031,  ..., 0.0024, 0.0020, 0.0021],
        [0.0022, 0.0028, 0.0033,  ..., 0.0021, 0.0036, 0.0019]],
       dtype=torch.float64)


tensor(0.0029, dtype=torch.float64)

In [59]:
def CRPS_calcul_interpolation_df(location,gev_stat,y_true):
    
        location = torch.tensor(location) 
        location = location.reshape(60, 60)
        #location = torch.transpose(location, 0,1)
        location = location.repeat(22, 1, 1)
        location = torch.permute(location, (1, 2, 0))
        

        c = torch.tensor(gev_stat['shape'])
        c = c.reshape(60, 60)
        #c = torch.transpose(c, 0,1) ##oujours 60,60
        c = c.repeat(22,1,1)
        c = torch.permute(c,(1,2,0))
        scale = torch.tensor(gev_stat['scale'])
        scale = scale.reshape(60, 60)
        #scale = torch.transpose(scale, 0, 1)  # Toujours 60, 60
        scale = scale.repeat(22, 1, 1)
        scale = torch.permute(scale, (1, 2, 0))

        size=(60,60,22)
        u = torch.rand(size)
            
        
        y_pred = location + (torch.pow(-torch.log(u), -c) - 1) * scale / c
        #print('mean y_pred',torch.mean(y_pred))
        #print('mean y_true',torch.mean(y_true))
        #print(y_true)
        term_one = torch.mean(torch.abs(y_pred - y_true), dim=-1)
        print(term_one)
        term_two = torch.mean(torch.abs(
        torch.unsqueeze(y_pred, -1) - torch.unsqueeze(y_pred, -2)), dim=(-2, -1))
        print(torch.mean(term_two))
        half = torch.tensor(-0.5, dtype=term_two.dtype)
        loss = term_one + half * term_two
        #print(loss)
        loss = torch.mean(loss)
        return loss

In [60]:
model_1_fitsp = pd.read_csv('model_1_fitspatgev.csv')
CRPS_calcul_interpolation_df(model_1_fitsp['loc'],gev_stat,out_ja)
model2 = pd.read_csv('model_2_vglm.csv')
CRPS_calcul_interpolation_df(model2['loc'],gev_stat,out_ja)
summer_basic = pd.read_csv('summer_basic_fitspatgev.csv')
CRPS_calcul_interpolation_df(summer_basic['loc'],gev_stat,out_ja)


tensor([[0.0034, 0.0033, 0.0040,  ..., 0.0045, 0.0089, 0.0142],
        [0.0036, 0.0057, 0.0034,  ..., 0.0096, 0.0043, 0.0028],
        [0.0031, 0.0045, 0.0046,  ..., 0.0213, 0.0107, 0.0055],
        ...,
        [0.0035, 0.0065, 0.0086,  ..., 0.0038, 0.0067, 0.0035],
        [0.0053, 0.0084, 0.0094,  ..., 0.0030, 0.0095, 0.0049],
        [0.0086, 0.0107, 0.0040,  ..., 0.0041, 0.0046, 0.0035]],
       dtype=torch.float64)
tensor(0.0065, dtype=torch.float64)
tensor([[0.0045, 0.0038, 0.0031,  ..., 0.0034, 0.0027, 0.0036],
        [0.0033, 0.0041, 0.0045,  ..., 0.0042, 0.0041, 0.0027],
        [0.0028, 0.0109, 0.0034,  ..., 0.0042, 0.0036, 0.0031],
        ...,
        [0.0069, 0.0058, 0.0063,  ..., 0.0037, 0.0047, 0.0063],
        [0.0046, 0.0035, 0.0058,  ..., 0.0043, 0.0066, 0.0028],
        [0.0088, 0.0043, 0.0044,  ..., 0.0040, 0.0047, 0.0036]],
       dtype=torch.float64)
tensor(0.0057, dtype=torch.float64)
tensor([[0.0037, 0.0035, 0.0038,  ..., 0.0049, 0.0041, 0.0029],
        [0.0

tensor(0.0031, dtype=torch.float64)

In [52]:
y_pred = out_ja
y_true = out_ja
term_one = torch.mean(torch.abs(y_pred - y_true), dim=-1)
print(term_one)
term_two = torch.mean(torch.abs(
torch.unsqueeze(y_pred, -1) - torch.unsqueeze(y_pred, -2)), dim=(-2, -1))
print(term_two)
print(torch.mean(term_two))
half = torch.tensor(-0.5, dtype=term_two.dtype)
loss = term_one + half * term_two
#print(loss)
loss = torch.mean(loss)
loss

IndentationError: unexpected indent (3043491426.py, line 12)